## Prepare and import necessary packages

In [ ]:
%pip install numpy matplotlib seaborn --upgrade

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math 

## Data about the planets in the solar system

In [ ]:
planets = {
           'Mercury':
           {'distance':0.39,'velocity':88/365,'mass':3.3e23},
           'Venus':
           {'distance':0.73,'velocity':225/365,'mass':4.9e24},
           'Earth':
           {'distance':1,'velocity':1,'mass':6e24},
           'Mars':
           {'distance':1.52,'velocity':687/365,'mass':6.6e23},
           'Jupiter':
           {'distance':5.2,'velocity':12,'mass':1.9e27},
           'Saturn':
           {'distance':9.54,'velocity':29.5,'mass':5.5e26},
           'Uranus':
           {'distance':19.19,'velocity':84,'mass':8.8e25},
           'Neptune':
           {'distance':30.06,'velocity':165,'mass':1.03e26},
          }



## Compute planetary positions as function of time

In [ ]:
t_tot = 20   # simulation time in years; complete orbit of Neptune takes 165 years

delta_t = .0001
n_steps = int(t_tot/delta_t)
n_planets = len(planets)

# position and velocity of each planet
pos = np.zeros((n_steps, n_planets, 2))
vel = np.zeros_like(pos)

# acceleration between planets (two iterations for velocity verlet method)
a = np.zeros((2, n_planets, n_planets, 2))

beta = 2.0  # exponent of energy potential

pos[0,:,0] = np.array([p['distance'] for p in planets.values()])
vel[0,:,1] = pos[0,:,0]*2*pi/np.array([p['velocity'] for p in planets.values()])

# masses
GM = (4*pi**2)/2e30 * np.array([p['mass'] for p in planets.values()])

for t in range(n_steps-1):
    for i in range(n_planets):
        a[0,i,i] = -4*pi**2*pos[t,i] / np.linalg.norm(pos[t,i],
                                                     axis = -1,
                                                     keepdims= True)**(beta+1)
        a[0,i+1:,i] = \
        a[0,i,i+1:] = -GM[i+1:].reshape(n_planets-i-1,1) \
                    * (pos[t,i+1:]-pos[t,i]) \
                    / np.linalg.norm(pos[t,i+1:]-pos[t,i],
                                     axis = -1,
                                     keepdims= True)**(beta+1)
        
        pos[t+1,i] = pos[t,i] + delta_t*vel[t,i] + delta_t**2/2 * np.sum(a[0,i])

        a[1,i,i] = -4*pi**2*pos[t+1,i] / np.linalg.norm(pos[t+1,i],
                                                       axis = -1,
                                                       keepdims= True)**(beta+1)
        a[1,i+1:,i] = \
        a[1,i,i+1:] = -GM[i+1:].reshape(n_planets-i-1,1) \
                    * (pos[t+1,i+1:]-pos[t+1,i]) \
                    / np.linalg.norm(pos[t+1,i+1:]-pos[t+1,i],
                                     axis = -1,
                                     keepdims= True)**(beta+1)
        vel[t+1,i] = vel[t,i] + delta_t*np.sum(a[:,i], axis = (0,1))/2


## Plot orbits

In [ ]:
sns.set()

radius = 2     # radius of observation (in AU)

plt.figure(figsize = (10,10))
for i,name in enumerate(planets):
    
    plt.plot(pos[:,i,0], pos[:,i,1], label = name)
    plt.xlim((-radius,radius))
    plt.ylim((-radius,radius))
    plt.ylabel("y position / AU")
    plt.xlabel("x position / AU")
    plt.legend()
    
plt.show()